In [1]:
#TDA API Greek Options Attribution Tool

### Data Aquisition

- Aquiring the data from TDA's API. 
- I'll have to use my TDA credentials, so I'm hiding them inside my env file and only using them as a specific variable. That way the actual credentials shouldn't be compromised.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

import env

import tda
import os
import requests
import json

Polygon.io API key loaded up.
Cboe API key loaded up.
TD Ameritrade API key loaded up.
Coinbase API key loaded up.
All credentials loaded successfully


In [4]:
tda_key = env.tda_api_key
tda_url = env.my_redirect_url

In [5]:
requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

<Response [200]>

In [6]:
aapl_api_test = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

In [ ]:
aapl_content = aapl_api_test.content

# aapl_content

In [24]:
request = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1').text
aapl = json.loads(request)
# print(aapl['callExpDateMap'])

In [25]:
aapl['callExpDateMap']

{'2021-03-05:15': {'129.0': [{'putCall': 'CALL',
    'symbol': 'AAPL_030521C129',
    'description': 'AAPL Mar 5 2021 129 Call (Weekly)',
    'exchangeName': 'OPR',
    'bid': 3.45,
    'ask': 3.6,
    'last': 3.5,
    'mark': 3.53,
    'bidSize': 13,
    'askSize': 15,
    'bidAskSize': '13X15',
    'lastSize': 0,
    'highPrice': 3.7,
    'lowPrice': 2.73,
    'openPrice': 0.0,
    'closePrice': 4.24,
    'totalVolume': 2066,
    'tradeDate': None,
    'tradeTimeInLong': 1613681974888,
    'quoteTimeInLong': 1613681999991,
    'netChange': -0.74,
    'volatility': 29.451,
    'delta': 0.552,
    'gamma': 0.05,
    'theta': -0.102,
    'vega': 0.106,
    'rho': 0.029,
    'openInterest': 3329,
    'timeValue': 2.79,
    'theoreticalOptionValue': 3.554,
    'theoreticalVolatility': 29.0,
    'optionDeliverablesList': None,
    'strikePrice': 129.0,
    'expirationDate': 1614978000000,
    'daysToExpiration': 15,
    'expirationType': 'S',
    'lastTradingDay': 1614992400000,
    'multi

In [26]:
aapl['callExpDateMap']['2021-03-05:15']['129.0'][0]

{'putCall': 'CALL',
 'symbol': 'AAPL_030521C129',
 'description': 'AAPL Mar 5 2021 129 Call (Weekly)',
 'exchangeName': 'OPR',
 'bid': 3.45,
 'ask': 3.6,
 'last': 3.5,
 'mark': 3.53,
 'bidSize': 13,
 'askSize': 15,
 'bidAskSize': '13X15',
 'lastSize': 0,
 'highPrice': 3.7,
 'lowPrice': 2.73,
 'openPrice': 0.0,
 'closePrice': 4.24,
 'totalVolume': 2066,
 'tradeDate': None,
 'tradeTimeInLong': 1613681974888,
 'quoteTimeInLong': 1613681999991,
 'netChange': -0.74,
 'volatility': 29.451,
 'delta': 0.552,
 'gamma': 0.05,
 'theta': -0.102,
 'vega': 0.106,
 'rho': 0.029,
 'openInterest': 3329,
 'timeValue': 2.79,
 'theoreticalOptionValue': 3.554,
 'theoreticalVolatility': 29.0,
 'optionDeliverablesList': None,
 'strikePrice': 129.0,
 'expirationDate': 1614978000000,
 'daysToExpiration': 15,
 'expirationType': 'S',
 'lastTradingDay': 1614992400000,
 'multiplier': 100.0,
 'settlementType': ' ',
 'deliverableNote': '',
 'isIndexOption': None,
 'percentChange': -17.36,
 'markChange': -0.71,
 'mar

In [17]:
aapl['callExpDateMap']['2021-03-05:15']['129.0']

[{'putCall': 'CALL',
  'symbol': 'AAPL_030521C129',
  'description': 'AAPL Mar 5 2021 129 Call (Weekly)',
  'exchangeName': 'OPR',
  'bid': 3.45,
  'ask': 3.6,
  'last': 3.5,
  'mark': 3.53,
  'bidSize': 13,
  'askSize': 15,
  'bidAskSize': '13X15',
  'lastSize': 0,
  'highPrice': 3.7,
  'lowPrice': 2.73,
  'openPrice': 0.0,
  'closePrice': 4.24,
  'totalVolume': 2066,
  'tradeDate': None,
  'tradeTimeInLong': 1613681974888,
  'quoteTimeInLong': 1613681999991,
  'netChange': -0.74,
  'volatility': 29.451,
  'delta': 0.552,
  'gamma': 0.05,
  'theta': -0.102,
  'vega': 0.106,
  'rho': 0.029,
  'openInterest': 3329,
  'timeValue': 2.79,
  'theoreticalOptionValue': 3.554,
  'theoreticalVolatility': 29.0,
  'optionDeliverablesList': None,
  'strikePrice': 129.0,
  'expirationDate': 1614978000000,
  'daysToExpiration': 15,
  'expirationType': 'S',
  'lastTradingDay': 1614992400000,
  'multiplier': 100.0,
  'settlementType': ' ',
  'deliverableNote': '',
  'isIndexOption': None,
  'percentCh

### Preparing the data

- The goal is to grab just the option greek and volatility data from the option chain. We don't need any other data at this point.
- I'm going to use a Pandas dataframe to help organize my work.

In [57]:
df = pd.DataFrame(aapl['callExpDateMap']['2021-03-05:15']['129.0'][0], index = [0])

df

,putCall,symbol,description,exchangeName,bid,ask,last,mark,bidSize,askSize,...,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,inTheMoney,mini,nonStandard
0,CALL,AAPL_030521C129,AAPL Mar 5 2021 129 Call (Weekly),OPR,3.45,3.6,3.5,3.53,13,15,...,100.0,,,None,-17.36,-0.71,-16.77,True,False,False


In [43]:
df.columns

Index(['putCall', 'symbol', 'description', 'exchangeName', 'bid', 'ask',
       'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize',
       'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume',
       'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange',
       'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest',
       'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility',
       'optionDeliverablesList', 'strikePrice', 'expirationDate',
       'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier',
       'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange',
       'markChange', 'markPercentChange', 'inTheMoney', 'mini', 'nonStandard'],
      dtype='object')

In [54]:
greeks = ['delta', 'gamma', 'theta', 'vega', 'rho']

volatility_info = ['openInterest', 'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility']

basic_info = ['mark', 'totalVolume', 'volatility', 'inTheMoney']

In [58]:
greeks + basic_info + volatility_info

['delta',
 'gamma',
 'theta',
 'vega',
 'rho',
 'mark',
 'totalVolume',
 'volatility',
 'inTheMoney',
 'openInterest',
 'timeValue',
 'theoreticalOptionValue',
 'theoreticalVolatility']

In [59]:
columns_list = basic_info + greeks + volatility_info

In [60]:
df[[c for c in df.columns if c in columns_list]]

,mark,totalVolume,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,inTheMoney
0,3.53,2066,29.451,0.552,0.05,-0.102,0.106,0.029,3329,2.79,3.554,29.0,True


In [61]:
df = df[[c for c in df.columns if c in columns_list]]
df

,mark,totalVolume,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,inTheMoney
0,3.53,2066,29.451,0.552,0.05,-0.102,0.106,0.029,3329,2.79,3.554,29.0,True


In [ ]:
# 3.53 to 2.50



In [ ]:
# https://www.livevol.com/apis/technical-reference/?m=market-at-a-glance/option-and-underlying-quotes